In [69]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests
import csv
import urllib.request
import time

In [70]:
## Creating a funtion that could manage different positions and areas
def get_url(position, area):
    templ = "https://www.indeed.co.uk/jobs?q={}&l={}"
    url = templ.format(position, area)
    return url    

In [71]:
## Here you can adapt the script to another search 
url = get_url("engineer", "London")
url

'https://www.indeed.co.uk/jobs?q=engineer&l=London'

## Extract the html

In [72]:
## Let's also use Selenium to open a web browser which will be helpfull for parsing the essential information 
browser = webdriver.Chrome()

In [73]:
## Get the URL and provide some seconds to the browser to load the page properly
browser.get(url)
time.sleep(2)

In [74]:
## Firstly, working alongside with the webpage, let's scrape all the job cards of the page
soup = bs(browser.page_source, "html.parser")
cards = soup.find_all("div", "jobsearch-SerpJobCard")
len(cards)

15

In [75]:
def get_features(card):
    try:
        atag = card.h2.a
        job_title = atag.get("title")
    except:
        job_title = None
    try:
        job_url = "http://www.indeed.com" + atag.get("href")
    except:
        job_url = None
    try:
        company_name = card.find("span", "company").text.strip()
    except:
        company_name = None
    try:
        location = card.find("div", "recJobLoc").get("data-rc-loc")
    except:
        location = None
    try:
        summary = card.find("div", "summary").text.strip()
    except:
        summary = None
    try:
        post_date = card.find("span", "date").text
    except:
        post_date = None
   
    ## Opening a new page in order to scrape the full job description

    try:
        new_urls = [x.find('h2', attrs={'class': 'title'}).find('a', href=True)['href'] for x in cards]
        new_urls = "http://www.indeed.com" + atag.get("href")
        page = urllib.request.urlopen(new_urls)
        soup = bs(page.read(), "html.parser")
        job_body = soup.find('div', attrs={'id': "jobDescriptionText"}).text.replace("\n", "")
    except:
        job_body = None


    ## Saving the features to a list    
    
    feature = (job_title, job_url, company_name, location, summary, post_date, job_body)
    return feature 
    

## Getting results from all the pages



In [76]:
features = []
i = 0
while i<2:
    try:
        url = 'https://www.indeed.co.uk' + soup.find("a", {"aria-label": "Next"}).get("href")
    except AttributeError:
         break
            
    response = requests.get(url)
    soup = bs(response.text, "html.parser")
    cards = soup.find_all("div", "jobsearch-SerpJobCard")
    for card in cards:
            feature = get_features(card)
            features.append(feature)
    i += 1
          

In [77]:
print(len(features))

30


In [78]:
## Convert the list to csv file
with open('data_set_engineer.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Job Title','Job Url', 'Company', 'Location','Summary', 'Posting Date', "Desc"])
        writer.writerows(features)

In [79]:
df = pd.read_csv("data_set_engineer.csv")
df.head()

,Job Title,Job Url,Company,Location,Summary,Posting Date,Desc
0,Graduate Mechanical Design Engineer,http://www.indeed.com/company/Panorama-Antenna...,Panorama Antennas Ltd,Wandsworth,This graduate mechanical design engineer posit...,Today,OpportunityThis is a fantastic opportunity to ...
1,Drainage Engineer,http://www.indeed.com/rc/clk?jk=c2833a00ab34da...,Lanes Group Plc,London,We require a drainage assistant to assist a dr...,30+ days ago,At the Lanes Group nothing is more important t...
2,Engineer,http://www.indeed.com/rc/clk?jk=f48e7d5b1bb0a4...,Bakkavor,London,4 on 4 off Shift Pattern – 18.30 – 06.30.\nIn ...,Just posted,Multi Skilled Engineer (Nights) – Harrow4 on 4...
3,Highways Engineer,http://www.indeed.com/rc/clk?jk=06c5a095fe9f22...,NSL Ltd,London,We offer a range of services focused on provid...,15 days ago,"Project Centre is a leading design, engineerin..."
4,Domestic Gas Engineer,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,Fixxa,South London,Spend less time travelling and more time on th...,Today,About FixxaFixxa is an innovative booking plat...
